In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from numpy import mean
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, GridSearchCV
#from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler


In [5]:
%store -r wp4
wp4 = wp4
 
%store -r wp8
wp8 = wp8

%store -r dwt4
dwt4 = dwt4

%store -r dwt8
dwt8 = dwt8

## Part A: Repeated Sampling Function

In [2]:
def tree_rep_samples(df, n_iter = 1000):
    df = df
    x = df.loc[:, df.columns != 'state'] #features
    y = df.loc[:, df.columns == 'state'] #supervisor

    eval_metrics = {} #empty dictionary to store metrics

    for i in range(n_iter): 

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.67)
        
        tree_model = RandomForestClassifier(class_weight='balanced')
        tree_model.fit(x_train, y_train.values.ravel())
        y_pred = tree_model.predict(x_test)

        acc = classification_report(y_test, y_pred, output_dict=True)['accuracy']
        spec = classification_report(y_test, y_pred, output_dict=True)['1']['precision'] #specificity
        sens = classification_report(y_test, y_pred, output_dict=True)['1']['recall'] #sensitivity

        metrics = [acc, spec, sens] #store values in list
        eval_metrics[i]=list(metrics) #store list in dictionary

    eval_metrics = pd.DataFrame.from_dict(eval_metrics).T 
    eval_metrics.columns = ['accuracy', 'specificity', 'sensitivity']
    eval_metrics.head()

    means = eval_metrics.mean(axis=0) #mean of each column
    
    return means

#### Wavelet Packet / Wang, 8-7-02 data (wp8)

In [6]:
tree_rep_samples(wp8, n_iter = 1000)

accuracy       0.935226
specificity    0.924784
sensitivity    0.978372
dtype: float64

#### Discrete Wavelet Transform, 8-7-02 data (dwt8)

In [7]:
tree_rep_samples(dwt8, n_iter = 1000)

accuracy       0.915655
specificity    0.907229
sensitivity    0.968078
dtype: float64